In [1]:
import numpy as np
from collections import defaultdict as dd
from typing import List, Dict, Set
import nltk
from tqdm import tqdm_notebook, tnrange
import pickle
from termcolor import colored

from enum import Enum

In [2]:
def load_word_set():
    all_lemmas = set()
    for line in open('Dane/polimorfologik-2.1.txt', encoding='utf-8'):
        L = line.split(';')[1].lower()
        all_lemmas.add(L)
    return all_lemmas

In [3]:
def get_normal_form(w):
    polish_map = {
        "ż": "z",
        "ź": "z",
        "x": "z",
        "ó": "o",
        "ł": "l",
        "ć": "c",
        "ą": "a",
        "ń": "n",
        "ę": "e",
    }
    ortographic_map = {
        "u": "o",
        "om": "a",
        "en": "e",
        "em": "e",
        "on": "a",
        "rz": "e",
        "ch": "h",
#         "fk": "wk",
#         "af": "aw",
        "f": "w",
        "sz": "z",
    }
    tmp = ""
    for c in w:
        tmp += c if c not in polish_map else polish_map[c]
    l = len(w)
    res = ""
    i = 0
    while i < l:
        if i < l - 1 and tmp[i:(i+2)] in ortographic_map:
            res += ortographic_map[tmp[i:(i+2)]]
            i += 2
            continue
        if tmp[i] in ortographic_map:
            res += ortographic_map[tmp[i]]
            i += 1
            continue
        res += tmp[i]
        i += 1
    return res

In [5]:
def get_dictionaries():
    ws = load_word_set()
    res1 = dd(list)
    res2 = dd(list)
    for w in word_set:
        norm_w = get_normal_form(w)
        res1[norm_w].append(w)
        res2[(w[0], len(w))].append(w)
        if len(w) > 1:
            res2[(w[1], len(w))].append(w)
    return res1, res2

In [4]:
class EditOp(Enum):
    INS = 1
    DEL = 2
    CHAN = 3
    SWAP = 4
    SWAP1 = 5

In [6]:
def levenshtein(seq1, seq2, ins_cost=1, del_cost=1, ch_cost=1, swp_cost=1, swp1_cost=1):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    matrix_op = [[[] for _ in range(size_y)] for _ in range(size_x)]
#     print(del_cost)
#     print(size_x)
    for x in range(size_x):
#         print(x)
        matrix[x, 0] = x*del_cost
        matrix_op[x][0] = [EditOp.DEL] * x
    for y in range(size_y):
        matrix[0, y] = y*ins_cost
        matrix_op[0][y] = [EditOp.INS] * y

#     print(matrix)
    for x in range(1, size_x):
        for y in range(1, size_y):
#             print(f"{x},{y}")
#             print(matrix[x-1, y] + del_cost)
            matrix[x,y] = matrix[x-1, y] + del_cost
            matrix_op[x][y] = matrix_op[x-1][y] + [EditOp.DEL]
            cost = matrix[x, y-1] + ins_cost
            if cost <= matrix[x, y]:
                matrix[x, y] = cost
                matrix_op[x][y] =  matrix_op[x][y-1] + [EditOp.INS]
            cost = matrix[x-1, y-1] if seq1[x-1] == seq2[y-1] else matrix[x-1, y-1] + ch_cost
            ops = matrix_op[x-1][y-1] if seq1[x-1] == seq2[y-1] else matrix_op[x-1][y-1] + [EditOp.CHAN]
            if cost <= matrix[x, y]:
                matrix[x, y] = cost
                matrix_op[x][y] =  ops
            if x > 1 and y > 1 and seq1[x-1] == seq2[y-2] and seq1[x-2] == seq2[y-1]:
                cost = matrix[x-2, y-2] + swp_cost
                ops = matrix_op[x-2][y-2] + [EditOp.SWAP]
                if cost <= matrix[x, y]:
                    matrix[x, y] = cost
                    matrix_op[x][y] =  ops
            if x > 2 and y > 2 and seq1[x-1] == seq2[y-3] and seq1[x-3] == seq2[y-1] and seq1[x-2] == seq2[y-2]:
                cost = matrix[x-2, y-2] + swp1_cost
                ops = matrix_op[x-2][y-2] + [EditOp.SWAP1]
                if cost <= matrix[x, y]:
                    matrix[x, y] = cost
                    matrix_op[x][y] =  ops
    return (matrix[size_x - 1, size_y - 1], matrix_op[size_x-1][size_y-1])

In [7]:
def argmin(l):
    res = None
    for i, el in enumerate(l):
        if res is None or el < l[res]:
            res = i
    return res
            

In [8]:
def find_closest(w, words, ins_cost=1, del_cost=1, ch_cost=1):
    edit_distances_and_ops = [levenshtein(w, w1, ins_cost, del_cost, ch_cost) for w1 in tqdm_notebook(words)]
    edit_distances = [el[0] for el in edit_distances_and_ops]
    i = argmin(edit_distances)
    return words[i], edit_distances_and_ops[i]

In [9]:
def correct_word(w, norm_dicts, ins_cost=1, del_cost=1, ch_cost=1):
    d1, d2 = norm_dicts
    norm_w = get_normal_form(w)
    if norm_w in d1:
        return find_closest(w, d1[norm_w], ins_cost, del_cost, ch_cost)
    l = len(w)
    w_candidates = []
    for i in range(l-1, l+2):
        w_candidates += d2[(w[0], i)][:]
    if len(w) > 1:
        for i in range(l -1, l + 2):
            w_candidates += d2[(w[1], i)]
    return find_closest(w, w_candidates, ins_cost, del_cost, ch_cost)

In [10]:
word_set = load_word_set()

In [11]:
d1, d2 = get_dictionaries()

In [ ]:
result = []
with open("Dane/literowki1.txt", "r") as ifile:
    for line in ifile:
        line = nltk.word_tokenize(line)
        correct, incorrect = line[0], line[1]
        corrected, ops = correct_word(line[1], (d1, d2))
        result.append((corrected, ops))
        print(f"{line[0]}, {line[1]} : {corrected}")

/home/mikolaj/miniconda3/envs/IoT-Pathway/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



lokomotywa, lokomowtuwa : lokomotywa



lokomotywa, kolokotywa : kolektiwa



lokomotywa, lokonowaywa : lokomotywa



lokomotywa, kolomotywa : lokomotywa



lokomotywa, lokotaywa : lokowała



prawdopodobieństwo, prawodpodoniestso : prawdopodobieństwo



prawdopodobieństwo, prawdopopdobieństwo : prawdopodobieństwo


In [ ]:
result[:2]

In [67]:
ops = []
with open("Dane/literowki1.txt", "r") as ifile:
    for i, line in enumerate(ifile):
        line = nltk.word_tokenize(line)
        correct, incorrect = line[0], line[1]
        ops += levenshtein(incorrect, correct)[1]

In [72]:
cnt1 = cnt2 = cnt3 = 0
for op in ops:
    if op == EditOp.INS:
        cnt1 += 1
    if op == EditOp.DEL:
        cnt2 += 1
    if op == EditOp.CHAN:
        cnt3 += 1
print(len(ops)/cnt1)
print(len(ops)/cnt2)
print(len(ops)/cnt3)

13.350877192982455
3.2382978723404254
1.6226012793176972


In [69]:
len(ops)

761

In [12]:
with open("error_correction_result1.pickle", "rb") as ifile:
    result1 = pickle.load(ifile)

In [38]:
with open("error_correction_result.pickle", "rb") as ifile:
    result = pickle.load(ifile)

In [41]:
all_c = 0
count = 0
with open("Dane/literowki1.txt", "r") as ifile:
    for i, line in enumerate(ifile):
        line = nltk.word_tokenize(line)
        correct, incorrect = line[0], line[1]
        print(f"{correct}: {result1[i][0]}")
        if correct == result1[i][0]:
            count += 1
        all_c += 1

lokomotywa: lokomotywa
lokomotywa: kolokowała
lokomotywa: lokomotywa
lokomotywa: lokomotywa
lokomotywa: lokowania
prawdopodobieństwo: prawdopodobieństw
prawdopodobieństwo: prawdopodobieństwo
prawdopodobieństwo: prawdopodobieństwo
prawdopodobieństwo: prawdopodobieństwom
prawdopodobieństwo: prawdopodobniejszej
prawdopodobieństwo: prawdopodobieństwom
prawdopodobieństwo: prawdopodobieństwo
prawdopodobieństwo: prawdopodobniejszej
prawdopodobieństwo: prawdopodobniejsi
prawdopodobieństwo: prawdopodobieństwo
prawdomówny: prawdomówno
prawdomówny: prawdomówny
prawdomówny: prawdzony
komputerek: dompteurek
komputerek: komputerem
komputerek: komputerek
komputerek: korektorek
komputerek: kompotierek
dziewczyna: dziewczyska
dziewczyna: dziewczyna
dziewczyna: dziewczynia
dziewczyna: dziewczynia
dziewczyna: dziewczyna
dziewczyna: dziewczyna
dziewczyna: dziedzina
dziewczyna: dziewczyn
dziewczyna: dziewczyn
przyjaźń: przyjeść
przyjaźń: przyjaźń
przyjaźń: przyjaźń
przyjaźń: przyjaźń
przyjaźń: przyjaźń
nis

In [42]:
count/all_c

0.44352617079889806

In [21]:
cntr = dd(lambda : 0)
total = 0
for el in result1:
    for op in el[1][1]:
#         print(el[1])
        cntr[op] +=1
        total +=1

In [25]:
print(f"Insertion: {cntr[EditOp.INS]/total}")
print(f"Deletion: {cntr[EditOp.DEL]/total}")
print(f"Change: {cntr[EditOp.CHAN]/total}")

Insertion: 0.002652519893899204
Deletion: 0.17639257294429708
Change: 0.8209549071618037


In [20]:
all_c = 0
count = 0
with open("Dane/literowki1.txt", "r") as ifile:
    for i, line in enumerate(ifile):
        line = nltk.word_tokenize(line)
        correct, incorrect = line[0], line[1]
        print(f"{correct}: {result[i][0]}")
        if correct == result[i][0]:
            count += 1
        all_c += 1

lokomotywa: lokomotywa
lokomotywa: kolektiwa
lokomotywa: lokomotywa
lokomotywa: lokomotywa
lokomotywa: lokowała
prawdopodobieństwo: prawdopodobieństwo
prawdopodobieństwo: prawdopodobieństwo
prawdopodobieństwo: prawdopodobieństwo
prawdopodobieństwo: prawdopodobieństwom
prawdopodobieństwo: prawdopodobieństwom
prawdopodobieństwo: prawdopodobieństwem
prawdopodobieństwo: prawdopodobieństwo
prawdopodobieństwo: prawdopodobieństwom
prawdopodobieństwo: prawdopodobieństw
prawdopodobieństwo: prawdopodobieństwo
prawdomówny: prawdomówno
prawdomówny: prawdomówny
prawdomówny: pradawny
komputerek: dompteurek
komputerek: komputerek
komputerek: komputerek
komputerek: korektorek
komputerek: komputerek
dziewczyna: dziewczyna
dziewczyna: dziewczyna
dziewczyna: dziewczynka
dziewczyna: dziewczyna
dziewczyna: dziewczyna
dziewczyna: dziewczyna
dziewczyna: dziewczyna
dziewczyna: dziewczyn
dziewczyna: dziewczyn
przyjaźń: przykaż
przyjaźń: przyjaźń
przyjaźń: przyjaźń
przyjaźń: przyjaźń
przyjaźń: przyjaźń
niszczyc

In [21]:
count

208

In [22]:
all_c

363

In [23]:
count/all_c

0.5730027548209367

In [36]:
cntr = dd(lambda : 0)
total = 0
for el in result:
    for op in el[1][1]:
#         print(el[1])
        cntr[op] +=1
        total +=1

In [37]:
print(f"Insertion: {cntr[EditOp.INS]/total}")
print(f"Deletion: {cntr[EditOp.DEL]/total}")
print(f"Change: {cntr[EditOp.CHAN]/total}")

Insertion: 0.047619047619047616
Deletion: 0.35154061624649857
Change: 0.6008403361344538


In [ ]:
result1 = []
with open("Dane/literowki1.txt", "r") as ifile:
    for line in ifile:
        line = nltk.word_tokenize(line)
        correct, incorrect = line[0], line[1]
        corrected, ops = correct_word(line[1], (d1, d2), ins_cost=13.350877192982455, del_cost=3.2382978723404254
, ch_cost=1.6226012793176972
)
        result1.append((corrected, ops))
        print(f"{line[0]}, {line[1]} : {corrected}")

/home/mikolaj/miniconda3/envs/IoT-Pathway/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



lokomotywa, lokomowtuwa : lokomotywa



lokomotywa, kolokotywa : kolokowała



lokomotywa, lokonowaywa : lokomotywa



lokomotywa, kolomotywa : lokomotywa



lokomotywa, lokotaywa : lokowania



prawdopodobieństwo, prawodpodoniestso : prawdopodobieństw



prawdopodobieństwo, prawdopopdobieństwo : prawdopodobieństwo



prawdopodobieństwo, prawdopodobieńśtwo : prawdopodobieństwo


In [ ]:
result2 = []
with open("Dane/literowki1.txt", "r") as ifile:
    for line in ifile:
        line = nltk.word_tokenize(line)
        correct, incorrect = line[0], line[1]
        corrected, ops = correct_word(line[1], (d1, d2), ins_cost=1, del_cost=1
, ch_cost=0.1
)
        result2.append((corrected, ops))
        print(f"{line[0]}, {line[1]} : {corrected}")

In [ ]:
import pickle
with open("error_correction_result.pickle", "wb") as ofile:
    pickle.dump(result, ofile)

In [ ]:
with open("error_correction_result1.pickle", "wb") as ofile:
    pickle.dump(result1, ofile)

In [17]:
"test"

'test'

In [19]:
results = [el[0] for el in result]